In [2]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs

#fd_set = open('C:\\Users\\non_coder\\Desktop\\datasets_fldr\\recipes_with_nutritional_info.json')
#data = json.load(fd_set)

In [3]:
d_set = pd.read_json("C:\\Users\\non_coder\\Desktop\\datasets_fldr\\recipes_with_nutritional_info.json")
#d_set.head(10)
#d_set = d_set[:100]
data_f = pd.DataFrame(d_set)
data_f = data_f.drop(['fsa_lights_per100g','instructions','quantity','url','weight_per_ingr','partition','nutr_per_ingredient','unit','id'], axis = 1)
data_f.head(5)

,ingredients,nutr_values_per100g,title
0,"[{'text': 'yogurt, greek, plain, nonfat'}, {'t...","{'energy': 81.12946131894766, 'fat': 2.1401392...",Yogurt Parfaits
1,"[{'text': 'sugars, granulated'}, {'text': 'oil...","{'energy': 477.09640393594606, 'fat': 23.41248...","Salt Free, Low Cholesterol Sugar Cookies Recipe"
2,"[{'text': 'chicken, broilers or fryers, wing, ...","{'energy': 208.05898280960727, 'fat': 14.29704...",Honey Sriracha Chicken Wings
3,"[{'text': 'crustaceans, shrimp, raw (not previ...","{'energy': 194.7525956849092, 'fat': 15.980767...",Shrimp and Caper Salad
4,"[{'text': 'cocoa, dry powder, unsweetened'}, {...","{'energy': 303.43539955190437, 'fat': 5.094846...",Natural Peanut Butter Chocolate Bon Bons


In [6]:
ingd = data_f['ingredients']

In [7]:
from pandas import DataFrame
import string
pd.set_option('display.max_colwidth', 100)
df = pd.DataFrame(d_set['ingredients'])
#print(df)
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
def remove_punctuation(txt):
    txt_nopunct = "".join([c for c in txt if c not in string.punctuation])
    return txt_nopunct

In [9]:
from string import digits
#d_set['ingredient'] = ingd.apply(lambda x: remove_punctuation(x))
#print(ingd[0])
#reslt_ingd = json.dumps(ingd[0])
res = []
for i in  ingd:
    res.append(json.dumps(i))
#res = remove_punctuation(reslt_ingd)
#res
#res_final = remove_punctuation(res)
#res_final

tmp = []
for i in res:
    i = i.replace('text','')
    tmp.append(remove_punctuation(i))
tmp
result = pd.DataFrame(tmp)
result
#remn = ''.join([i for i in result if not i.isdigit()])
data_f['cleaned'] = result

In [10]:
data_f['cleaned'] = data_f['cleaned'].str.replace('\d+', '')
d_table = pd.DataFrame(data_f)

In [12]:
#d_table.to_csv ('C:\\Users\\non_coder\\Desktop\\datasets_fldr\\exported_data.csv', index = False, header=True)
d_table = pd.DataFrame(d_table)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(max_df=0.8, max_features=200000, min_df=0.2, analyzer = 'word',ngram_range = (1,4),stop_words = 'english')

In [15]:
fd_tidf = tf.fit_transform(d_table['cleaned'])
fd_tidf.shape

(51235, 35)

In [16]:
terms = tf.get_feature_names()
terms

['allpurpose',
 'allpurpose unenriched',
 'bottled',
 'bottled generic',
 'butter',
 'butter salt',
 'flour',
 'flour white',
 'flour white allpurpose',
 'flour white allpurpose unenriched',
 'fluid',
 'generic',
 'granulated',
 'milk',
 'oil',
 'powder',
 'raw',
 'salad',
 'salt',
 'salt table',
 'spices',
 'sugars',
 'sugars granulated',
 'table',
 'unenriched',
 'water',
 'water bottled',
 'water bottled generic',
 'wheat',
 'wheat flour',
 'wheat flour white',
 'wheat flour white allpurpose',
 'white',
 'white allpurpose',
 'white allpurpose unenriched']

In [28]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(fd_tidf)

In [29]:
from sklearn.cluster import KMeans

num_clusters = 10

km = KMeans(n_clusters=num_clusters)

%time km.fit(fd_tidf)

clusters = km.labels_.tolist()

Wall time: 6.62 s


In [32]:
clusters = km.labels_.tolist()

In [34]:
data_f['title'].value_counts()

Oatmeal Lace Cookies                      1
Easy Cheese Sauce                         1
Oatmeal Fiber One Berry Breakfast Bars    1
Cherry Tomato Corn Salad                  1
Peanut Brittle                            1
                                         ..
Hearty Cheese (And Turkey) Soup           1
Yin-Yang Soup                             1
Buttermilk Chocolate Biscuits             1
Frozen Banana Smoothie                    1
Chocolate Chip Ice Cream                  1
Name: title, Length: 100, dtype: int64

In [39]:
grouped = data_f['terms'].groupby(data_f['title'])

grouped.mean()

KeyError: 'terms'